# Limpieza y transformación

##### Integrantes: 
##### - Harold Aquino Curisinche 
##### - Rosa Estrada Estrada
##### Descripción: Limpieza y transformación del dataset de reclamos de SUSALUD

## 1. Importación de librerías

In [2]:
import pandas as pd
import numpy as np

### Cargar datos procesados

In [98]:
df = pd.read_csv("../data/processed/reclamos_clean.csv", low_memory=False)

In [99]:
# Mostrar primeras filas
df.head()

,ID_PERIODO,DE_TIPO_ADMINISTRADO,CO_ADMIN_DECLA,CO_UGIPRESS,DE_TIPO_INSTITUCION,CO_ADMIN_SUCE,DE_MEDIO_PRESENTACION,CO_UNICO_RECLAMO,DE_MEDIO_RECEPCION,FE_PRESEN_RECLA,...,DE_ETAPA_RECLAMO,DE_TIPO_ADMIN_DERIVA,CO_ADMIN_DERIVA,DE_RESULTADO,DE_MOTIVO_CONCL_ANTIC,FE_RESULT_RECL,DE_COMUN_RESULT,FE_NOTIFI_RESULT,AÑO,MES
0,202101,IAFAS,20007,20007,IAFAS,20007,Físico,20007-352,Libro de Reclamaciones Físico,2020-11-27,...,Resultado y Notificación,-,-,Infundado,-,2020-12-21,Correo electrónico,NaN,2020,11
1,202101,IAFAS,20007,20007,IAFAS,20007,Físico,20007-360,Libro de Reclamaciones Físico,2021-01-04,...,Resultado y Notificación,-,-,Infundado,-,2021-02-18,Correo electrónico,NaN,2021,1
2,202101,IAFAS,20007,20007,IAFAS,20007,Físico,20007-361,Libro de Reclamaciones Físico,2021-01-06,...,Archivo y Custodia del Expediente,-,-,Infundado,-,2021-01-20,Correo electrónico,20210122.0,2021,1
3,202101,IAFAS,20007,20007,IAFAS,20007,Físico,20007-364,Libro de Reclamaciones Físico,2021-01-06,...,Archivo y Custodia del Expediente,-,-,Infundado,-,2021-01-25,Correo electrónico,20210204.0,2021,1
4,202101,IAFAS,20007,20007,IAFAS,20007,Físico,20007-365,Libro de Reclamaciones Físico,2021-01-08,...,Evaluación e investigación,-,-,Pendiente,-,NaN,-,NaN,2021,1


In [105]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162639 entries, 0 to 162638
Data columns (total 22 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   ID_PERIODO             162639 non-null  int64 
 1   DE_TIPO_ADMINISTRADO   162639 non-null  object
 2   DE_TIPO_INSTITUCION    162639 non-null  object
 3   DE_MEDIO_PRESENTACION  162639 non-null  object
 4   CO_UNICO_RECLAMO       162639 non-null  object
 5   DE_MEDIO_RECEPCION     162639 non-null  object
 6   FE_PRESEN_RECLA        162639 non-null  object
 7   DE_SERVICIO            162639 non-null  object
 8   DE_COMPETENCIA         162639 non-null  object
 9   DE_CLASIF_1            162639 non-null  object
 10  DE_CLASIF_2            162639 non-null  object
 11  DE_CLASIF_3            162639 non-null  object
 12  DE_ESTADO_RECLAMO      162639 non-null  object
 13  DE_ETAPA_RECLAMO       162639 non-null  object
 14  DE_TIPO_ADMIN_DERIVA   162639 non-null  object
 15  

In [101]:
# Identificamos columnas irrelevantes para el análisis
columns_to_drop = ["CO_ADMIN_DECLA", "CO_UGIPRESS", "CO_ADMIN_SUCE", 
                   "CO_RECLAMO_PRIMIG", "CO_ADMIN_DERIVA"]

# Eliminamos esas columnas
df.drop(columns=columns_to_drop, inplace=True)

In [106]:
# Convertir fechas a formato datetime
df["FE_PRESEN_RECLA"] = pd.to_datetime(df["FE_PRESEN_RECLA"], format="%Y-%m-%d", errors="coerce")
df["FE_RESULT_RECL"] = pd.to_datetime(df["FE_RESULT_RECL"], format="%Y-%m-%d", errors="coerce")
df["FE_NOTIFI_RESULT"] = pd.to_datetime(df["FE_NOTIFI_RESULT"], format="%Y-%m-%d", errors="coerce")

In [107]:
# Extraer el año y mes para facilitar análisis
df["AÑO"] = df["FE_PRESEN_RECLA"].dt.year
df["MES"] = df["FE_PRESEN_RECLA"].dt.month

In [109]:
# Ver columnas con valores nulos
df.isnull().sum()

# Imputar valores nulos en columnas de texto con "Desconocido"
cols_texto = ["DE_TIPO_ADMINISTRADO", "DE_TIPO_INSTITUCION", "DE_MEDIO_PRESENTACION", 
              "DE_MEDIO_RECEPCION", "DE_SERVICIO", "DE_CLASIF_1", "DE_CLASIF_2", 
              "DE_CLASIF_3", "DE_ESTADO_RECLAMO", "DE_ETAPA_RECLAMO", "DE_RESULTADO"]
df[cols_texto] = df[cols_texto].fillna("Desconocido")

# Imputar valores nulos en fechas con la fecha mínima
df = df.assign(
    FE_RESULT_RECL=df["FE_RESULT_RECL"].fillna(df["FE_PRESEN_RECLA"]),
    FE_NOTIFI_RESULT=df["FE_NOTIFI_RESULT"].fillna(df["FE_RESULT_RECL"])
)

In [110]:
df['FE_PRESEN_RECLA'] = pd.to_datetime(df['FE_PRESEN_RECLA'])
df['FE_RESULT_RECL'] = pd.to_datetime(df['FE_RESULT_RECL'])

In [111]:
# Calcular el tiempo de resolución de los reclamos
df["DURACION_RECLAMO"] = (df["FE_RESULT_RECL"] - df["FE_PRESEN_RECLA"]).dt.days

# Clasificar la duración en categorías
df["CATEGORIA_DURACION"] = pd.cut(df["DURACION_RECLAMO"], bins=[-1, 7, 30, 90, 365, np.inf], 
                                  labels=["Corto (0-7 días)", "Medio (8-30 días)", 
                                          "Largo (31-90 días)", "Muy Largo (91-365 días)", 
                                          "Extremo (>365 días)"])


In [114]:
# Guardamos el nuevo dataset limpio
df.to_csv("../data/processed/reclamos_clean_transformed.csv", index=False, encoding="utf-8")

PermissionError: [Errno 13] Permission denied: '../data/processed/reclamos_clean_transformed.csv'